# Langchain crash course

In [ ]:
# pip install langchain openai serpapi langchain-community langchain-openai google-search-results numexpr wikipedia
# !pip install langchain openai serpapi langchain-community langchain-openai google-search-results numexpr wikipedia

In [33]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

# print(openapi_key)

## LLMs

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\380268205.py:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9)
C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\380268205.py:4: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")



"Saffron Palace"


In [3]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\2943582005.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")


'\n\n"Royal Spice Bistro"'

## Prompt Templates

In [4]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\723744833.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template_name)
C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\723744833.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Mexican")


'\n\n"El Cocinero Mexicano" (The Mexican Chef)'

In [6]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n"Casa del Sabor" (House of Flavor)'

In [7]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [8]:
# it will run the first chain and then pass the output to the second chain and return the final output
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Chicken Tikka Masala: Tender pieces of chicken in a rich and creamy tomato-based sauce, served with basmati rice and naan bread.

2. Vegetable Biryani: Fragrant basmati rice cooked with mixed vegetables, spices, and herbs, served with raita (yogurt sauce).

3. Lamb Rogan Josh: Slow-cooked lamb in a flavorful sauce made with onions, tomatoes, and a blend of spices, served with rice or naan.

4. Palak Paneer: Creamy spinach and cottage cheese curry, served with basmati rice and garlic naan.

5. Tandoori Chicken: Marinated chicken cooked in a clay oven, served with basmati rice, vegetables, and mint chutney.

6. Chana Masala: A popular vegetarian dish made with chickpeas, onions, tomatoes, and spices, served with rice or naan.

7. Butter Chicken: Tender pieces of chicken cooked in a rich, buttery tomato-based sauce, served with basmati rice and garlic naan.

8. Dal Makhani: A creamy and flavorful lentil dish made with black lentils, kidney beans, and spices, served with basmati rice 

#### Sequential Chain

In [9]:
# Now we will use the same llm and prompt template to create a new chain
# so we don't have to create a new llm and prompt template for every chain
# we can reuse them and will provide all the outputs requested that are activing as input to the next chain.
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [10]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [11]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [12]:
chain({"cuisine": "Indian"})

C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\3052807682.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"cuisine": "Indian"})


{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Maharaja\'s Palace: A Taste of India"',
 'menu_items': '\n\n1. Tandoori Chicken: Marinated chicken cooked in a clay oven, served with a side of naan bread.\n2. Vegetable Biryani: Basmati rice cooked with mixed vegetables, aromatic spices, and nuts.\n3. Butter Chicken: Tender chicken pieces cooked in a rich and creamy tomato-based sauce.\n4. Palak Paneer: Cubes of Indian cottage cheese cooked in a creamy spinach sauce.\n5. Lamb Rogan Josh: Slow-cooked lamb in a flavorful gravy of tomatoes, onions, and spices.\n6. Chana Masala: Chickpeas cooked in a spicy tomato and onion gravy.\n7. Samosas: Fried pastry filled with potatoes, peas, and spices.\n8. Chicken Tikka Masala: Grilled chicken cooked in a creamy tomato and onion sauce.\n9. Aloo Gobi: Cauliflower and potatoes cooked with spices and herbs.\n10. Mango Lassi: A refreshing yogurt-based drink with mango puree and spices.'}

## Agents

In [ ]:
# make sure yoy have installed this package: pip install google-search-results
from secret_key import serpapi_key
os.environ['SERPAPI_API_KEY'] = serpapi_key


#### serpapi and llm-math tool

In [14]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\417093560.py:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)




> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: 2022 + 5
Observation: Answer: 2027
Thought: Now I need to search for the GDP of the US in 2027.
Action: Search
Action Input: "GDP of US in 2027"
Observation: ['In the long-term, the United States GDP is projected to trend around 30156.00 USD Billion in 2026 and 30729.00 USD Billion in 2027, according to our econometric ...', 'The initial read of real GDP growth for 2025Q1 indicated a contraction of 0.3 percent at an annual rate. Chart 1 shows the contributions of ...', 'Release Table for 2027, Release Tables: Summary of Economic Projections. FRED: Download, graph, and track economic data.', 'https://www.cbo.gov/publication/52370', 'According to the World Bank, the U.S. accounted for 14.8% of the global aggregate GDP in 2024 in purchasing power parity terms and 26.2% in nominal terms. The ...', 'CBO expects real (inflation-adjusted) GDP to grow by 2.3 pe

'The projected GDP of the US in 2027 is around 30729.00 USD Billion.'

#### Wikipedia and llm-math tool

In [15]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk  ( EE-lon; born June 28, 1971) is a businessman. He is known for his leadership of Tesla, SpaceX, X (formerly Twitter), and the Department of Government Efficiency (DOGE). Musk has been considered the wealthiest person in the world since 2021; as of May 2025, Forbes estimates his net worth to be US$424.7 billion. 
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He received bachelor's degrees from the University of Pennsylvania in 1997 before moving to California, United States, to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also became an Ameri

'Elon Musk was born on June 28, 1971 and his age in 2023 would be 52 years old.'

## Memory

In [16]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [17]:
name = chain.run("Indian")
print(name)



"Spice Palace"


In [18]:
chain.memory

In [19]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [20]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\3140656249.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()




"El Sabor Exquisito" (The Exquisite Flavor)


In [21]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [22]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [23]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


C:\Users\erdkg\AppData\Local\Temp\ipykernel_17644\3006166724.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  convo = ConversationChain(llm=OpenAI(temperature=0.7))


In [24]:
convo.run("Who won the first cricket world cup?")

" The first cricket world cup was held in 1975 and was won by the West Indies. However, back then it was called the Prudential Cup. The West Indies beat Australia by 17 runs in the final match at Lord's Cricket Ground in London. The players of the West Indies team at that time were Clive Lloyd, Viv Richards, Gordon Greenidge, and Michael Holding, just to name a few. It was a historic moment for cricket and the West Indies team dominated the sport for years to come."

In [25]:
convo.run("How much is 5+5?")

' 5+5 is equal to 10. This is a basic mathematical equation that follows the principles of addition, where two numbers are added together to get a sum. The concept of addition is a fundamental mathematical concept and is used in various fields such as finance, science, and engineering. It is also commonly taught in schools as part of basic arithmetic.'

In [26]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning West Indies team in the 1975 cricket world cup was Clive Lloyd. He led the team to victory and was also the highest run-scorer in the final match, scoring 102 runs. Lloyd was known for his aggressive batting style and strong leadership skills, which helped the West Indies team become one of the most dominant teams in cricket during that time. He is considered a legend in the sport and was later inducted into the ICC Cricket Hall of Fame in 2009.'

In [27]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies. However, back then it was called the Prudential Cup. The West Indies beat Australia by 17 runs in the final match at Lord's Cricket Ground in London. The players of the West Indies team at that time were Clive Lloyd, Viv Richards, Gordon Greenidge, and Michael Holding, just to name a few. It was a historic moment for cricket and the West Indies team dominated the sport for years to come.
Human: How much is 5+5?
AI:  5+5 is equal to 10. This is a basic mathematical equation that follows the principles of addition, where two numbers are added together to get a sum. The concept of addition is a fundamental mathematical concept and is used in various fields such as finance, science, and engineering. It is also commonly taught in schools as part of basic arithmetic.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning West Indies team in the

#### ConversationBufferWindowMemory

In [29]:
# Now we will use a memory that only keeps the last 1 interaction
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and was won by the West Indies team. They defeated Australia by 17 runs in the final match. The captain of the West Indies team at the time was Clive Lloyd and the top scorer for the team in the final match was Viv Richards with 138 runs. The tournament was hosted by England and was played in a 60-overs-per-side format.'

In [30]:
convo.run("How much is 5+5?")

' 5+5 is equal to 10.'

In [31]:
convo.run("Who was the captain of the winning team?")

' I do not have enough context to accurately answer that question. Do you have any specific team or event in mind?'